In [ ]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore') 

import tensorflow.keras as keras
import tensorflow as tf
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import random
from keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.layers import Conv2D, Activation,AveragePooling2D,MaxPooling2D,GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import Adam

In [ ]:
path_to_train_wav='/content/drive/MyDrive/prism_dataset/train_wav'
path_to_test_wav='/content/drive/MyDrive/prism_dataset/test_wav'

path_to_train_img='/content/drive/MyDrive/prism_dataset/train_img'
path_to_test_img='/content/drive/MyDrive/prism_dataset/test_img'

In [ ]:
import skimage.io

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

In [4]:
# FOR Training wav to images
for filename in os.listdir(path_to_train_wav):
    wavefile = path_to_train_wav  + '/' + str(filename)
    y, sr = librosa.load(wavefile, mono=True,duration=1,sr=16000)
    N = 16000-len(y)
    y=np.pad(y, (0, N), 'constant')
    mfcc = librosa.feature.melspectrogram(y=y, sr=sr)
    
    img = scale_minmax(mfcc, 0, 255).astype(np.uint8) 
    
    skimage.io.imsave(path_to_train_img + '/' + str(filename[:-4]) + '.png', img)




# # FOR Testing wav to images
# for filename in os.listdir(pathtotestwav):
#     songname = pathtotestwav  + '/' + str(filename)
#     y, sr = librosa.load(songname, mono=True,duration=1,sr=16000)
#     N = 16000-len(y)
#     y=np.pad(y, (0, N), 'constant')
#     mfcc = librosa.feature.mfcc(y=y, sr=sr)
   
#     img = scale_minmax(mfcc, 0, 255).astype(np.uint8)
   
#     skimage.io.imsave(pathtotestimg + '/' + str(filename[:-4]) + '.png', img)

In [5]:
import cv2
from google.colab.patches import cv2_imshow
labels = 'abhishek anmol anurag deewanshu himanshu kishan mayank narender neeraj prince ritvik sayan shiv shreyas shubham siddharth sourav swaraj utkarsh vikrant'.split()


In [6]:
# FOR Training images
train_data = []
train_label = []
j=1

for photos in os.listdir(path_to_train_img):
  image_array=cv2.imread(path_to_train_img + '/' + str(photos),0)
  # if j==1:
  #   cv2_imshow(image_array)
  #   j+=1
  image_array = np.expand_dims(image_array, axis=-1)
  # image_array=cv2.imread(f'./drive/MyDrive/dataset_prism/dataset_img/train_img/{photos}')
  # image_array=Image.open(f'./drive/MyDrive/dataset_img/{photos}')
  # image_array = np.asarray(image_array)
  train_data.append(image_array)
  for temp in range(len(labels)):
    if labels[temp]==photos[:-7]:
      train_label.append(temp)
      break


#Converting training images to training arrays
train_data = np.asarray(train_data)
train_label = np.asarray(train_label)
print(train_data.shape)
print(train_label.shape)

(1000, 128, 32, 1)
(1000,)


In [7]:
# # FOR Testing images
# test_data = []
# test_label = []
#                                     # Files = ['Dog', 'Cat']
#                                     # label_val = 0
# for photos in os.listdir(pathtotestimg):
#                                     # print(photos)
#   image_array=cv2.imread(pathtotestimg + '/' + str(photos))
#   image_array = np.expand_dims(image_array, axis=-1)
#                                     # image_array=Image.open(f'./drive/MyDrive/dataset_img/{photos}')
#                                     # image_array = np.asarray(image_array)
#   test_data.append(image_array)
#   for temp in range(len(labels)):
#     if labels[temp]==photos[:-7]:
#       test_label.append(temp)
#       break
  
#                                     #Converting testing images to testing arrays
# test_data = np.asarray(test_data)
# test_label = np.asarray(test_label)
# print(test_data.shape)
# print(test_label.shape)

In [14]:
x_train=train_data
y_train=train_label
# X_test=test_data
# y_test=test_label
print(x_train.shape)
print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)

(1000, 128, 32, 1)
(1000,)


In [15]:
num_classes = 20
epochs = 20
 
def euclid_dis(vects):
  x,y = vects
  sum_square = K.sum(K.square(x-y), axis=1, keepdims=True)
  return K.sqrt(K.maximum(sum_square, K.epsilon()))
 
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)
 
def contrastive_loss(y_true, y_pred):
    y_true=tf.dtypes.cast(y_true, tf.float64)
    y_pred=tf.dtypes.cast(y_pred, tf.float64)
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [16]:
# PYIMAGESEARCH
def create_pairs(images, labels):
	# initialize two empty lists to hold the (image, image) pairs and
	# labels to indicate if a pair is positive or negative
	pairImages = []
	pairLabels = []
  # calculate the total number of classes present in the dataset
	# and then build a list of indexes for each class label that
	# provides the indexes for all examples with a given label
	numClasses = len(np.unique(labels))
	idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
  # loop over all images
	for idxA in range(len(images)):
		# grab the current image and label belonging to the current
		# iteration
		currentImage = images[idxA]
		label = labels[idxA]
		# randomly pick an image that belongs to the *same* class
		# label
		idxB = np.random.choice(idx[label])
		posImage = images[idxB]
		# prepare a positive pair and update the images and labels
		# lists, respectively
		pairImages.append([currentImage, posImage])
		pairLabels.append(1)
    # grab the indices for each of the class labels *not* equal to
		# the current label and randomly pick an image corresponding
		# to a label *n ot* equal to the current label
		negIdx = np.where(labels != label)[0]
		negImage = images[np.random.choice(negIdx)]
		# prepare a negative pair of images and update our lists
		pairImages.append([currentImage, negImage])
		pairLabels.append(0)
	# return a 2-tuple of our image pairs and labels
	return (np.array(pairImages), np.array(pairLabels))

In [17]:
def create_base_net(input_shape):
   
  input = Input(shape = input_shape)
  x = Conv2D(4, (5,5), activation = 'tanh')(input)
  x = AveragePooling2D(pool_size = (2,2))(x)
  x = Conv2D(16, (5,5), activation = 'tanh')(x)
  x = AveragePooling2D(pool_size = (2,2))(x)
  x = Flatten()(x)
  x = Dense(20, activation = 'tanh')(x)
  model = Model(input, x)
  model.summary()
  return model
 
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)
 
 
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))
 
 
# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
 
 
# x_train = x_train.reshape(x_train.shape[0], 28, 28,1)
# x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (128, 32,1)
print(x_train.shape)
x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# y_train = y_train.astype('float32')
# y_test = y_test.astype('float32')
x_train /= 255
# x_test /= 255
 
input_shape = x_train.shape[1:]
input_shape = (128, 32, 1)

(1000, 128, 32, 1)


In [18]:
# digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, y_train)

In [19]:
base_network = create_base_net(input_shape)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 32, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 124, 28, 4)        104       
_________________________________________________________________
average_pooling2d (AveragePo (None, 62, 14, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 10, 16)        1616      
_________________________________________________________________
average_pooling2d_1 (Average (None, 29, 5, 16)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2320)              0         
_________________________________________________________________
dense (Dense)                (None, 20)                46420 

In [20]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
 
processed_a = base_network(input_a)
processed_b = base_network(input_b)
 
distance = Lambda(euclid_dis,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])
 
model = Model([input_a, input_b], distance)
#train
model.compile(loss=contrastive_loss, optimizer='adam', metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=20)

Epoch 1/20
16/16 [==============================] - 8s 390ms/step - loss: 0.3496 - accuracy: 0.5105
Epoch 2/20
16/16 [==============================] - 6s 395ms/step - loss: 0.2477 - accuracy: 0.5953
Epoch 3/20
16/16 [==============================] - 6s 391ms/step - loss: 0.2279 - accuracy: 0.6253
Epoch 4/20
16/16 [==============================] - 6s 390ms/step - loss: 0.2179 - accuracy: 0.6454
Epoch 5/20
16/16 [==============================] - 6s 392ms/step - loss: 0.2108 - accuracy: 0.6729
Epoch 6/20
16/16 [==============================] - 6s 389ms/step - loss: 0.2058 - accuracy: 0.6893
Epoch 7/20
16/16 [==============================] - 6s 391ms/step - loss: 0.2019 - accuracy: 0.6997
Epoch 8/20
16/16 [==============================] - 6s 389ms/step - loss: 0.1976 - accuracy: 0.7089
Epoch 9/20
16/16 [==============================] - 6s 388ms/step - loss: 0.1950 - accuracy: 0.7149
Epoch 10/20
16/16 [==============================] - 6s 389ms/step - loss: 0.1921 - accuracy: 0.7227

In [31]:
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])


In [32]:
y_pred[:9]

array([[3.0170438e-01],
       [6.6186249e-01],
       [5.0869775e-01],
       [7.0596337e-01],
       [2.8502491e-01],
       [7.6076597e-01],
       [2.2653650e-01],
       [6.1600655e-01],
       [3.1622776e-04]], dtype=float32)

In [33]:
tr_y[10:20]

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [34]:
y_pred = y_pred.ravel() < 0.5
y_pred[10:20]

array([ True,  True, False, False,  True, False,  True, False,  True,
       False])

In [25]:
import sklearn
from sklearn.metrics import roc_auc_score, matthews_corrcoef
print(sklearn.metrics.matthews_corrcoef(tr_y, y_pred))

0.5351672659099802


In [26]:
print(sklearn.metrics.roc_auc_score(tr_y, y_pred))

0.7675000000000001
